## Imports

In [ ]:
import os
import cv2
import torch
import shutil
import random
import tempfile
import numpy as np
from PIL import Image
from skimage import io
from pathlib import Path
from monai.transforms import LoadImage
from pytorch_fid import fid_score
from torchvision import transforms
from pytorch_msssim import ssim

## Divide Real Images in Half

In [ ]:
'''
from glob import glob

def split_images(src_folder, dest_folder1, dest_folder2):
    # Create destination folders if they don't exist
    os.makedirs(dest_folder1, exist_ok=True)
    os.makedirs(dest_folder2, exist_ok=True)
    
    # Get list of image files in the source folder
    image_files = glob(os.path.join(src_folder, '*.png'))
    
    # Split the image files list in half
    midpoint = len(image_files) // 2
    first_half = image_files[:midpoint]
    second_half = image_files[midpoint:]
    
    # Copy first half to the first destination folder
    for image_file in first_half:
        shutil.copy(image_file, dest_folder1)
    for image_file in second_half:
        shutil.copy(image_file, dest_folder2)
        
    print(f'{len(first_half)} images copied to {dest_folder1}')
    print(f'{len(second_half)} images copied to {dest_folder2}')

# Example usage:
src_folder = './real_images/275'
dest_folder1 = './real_images/275_1'
dest_folder2 = './real_images/275_2'

split_images(src_folder, dest_folder1, dest_folder2)
'''

## FID Calculation

In [ ]:
### Real
path_real = "./real_images/275"
path_real_1 = "./real_images/275_1"
path_real_2 = "./real_images/275_2"
fid_value_real_real = fid_score.calculate_fid_given_paths([path_real_1, path_real_2], batch_size=60, device='cuda', dims=768)
print("FID real vs real:", fid_value_real_real)

### FID 7.0
path_generated = "./generated_images/7.0"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=60, device='cuda', dims=768)
print("FID_7.0 real vs generated:", fid_value_real_generated)

### FID 5.0
path_generated = "./generated_images/5.0"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=60, device='cuda', dims=768)
print("FID_5.0 real vs generated:", fid_value_real_generated)

### FID 3.0
path_generated = "./generated_images/3.0"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=60, device='cuda', dims=768)
print("FID_3.0 real vs generated:", fid_value_real_generated)

### FID 1.0
path_generated = "./generated_images/1.0"
fid_value_real_generated = fid_score.calculate_fid_given_paths([path_real, path_generated], batch_size=60, device='cuda', dims=768)
print("FID_1.0 real vs generated:", fid_value_real_generated)

## SSIM Calculation

In [ ]:
def load_images_from_folder(folder, device='cuda'):
    images = [] 
    transform = transforms.Compose([
        transforms.ToTensor()  # Convert image to tensor [0,1] range
    ])
    
    for filename in sorted(os.listdir(folder)):
        if filename.endswith('.png'):
            file_path = os.path.join(folder, filename)
            img = cv2.imread(file_path, cv2.IMREAD_GRAYSCALE) 
            img = transform(img).unsqueeze(0).to(device)
            images.append(img)
    return images

def calculate_ssim(images):
    ssim_values = []
    image_count = len(images)
    
    # Generate all possible pairs without replacement
    indices = list(range(image_count))
    random.shuffle(indices)

    for i in range(image_count):
        for j in range(i + 1, image_count):
            img1 = images[indices[i]]
            img2 = images[indices[j]]

            # Compute SSIM
            ssim_value = ssim(img1, img2, data_range=1.0)
            ssim_values.append(ssim_value.item())

    return ssim_values

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [ ]:
### Real
path_real = "./real_images/275"
real_images = load_images_from_folder(path_real, device)
ssim_values_real_real = calculate_ssim(real_images)
print("Average SSIM real vs real:", np.mean(ssim_values_real_real))

### SSIM 7.0
path_generated = "./generated_images/7.0"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 7.0 generated vs generated:", np.mean(ssim_values_gen_gen))

### SSIM 5.0
path_generated = "./generated_images/5.0"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 5.0 generated vs generated:", np.mean(ssim_values_gen_gen))

### SSIM 3.0
path_generated = "./generated_images/3.0"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 3.0 generated vs generated:", np.mean(ssim_values_gen_gen))

### SSIM 1.0
path_generated = "./generated_images/1.0"
generated_images = load_images_from_folder(path_generated)
ssim_values_gen_gen = calculate_ssim(generated_images)
print("Average SSIM 1.0 generated vs generated:", np.mean(ssim_values_gen_gen))